<a href="https://colab.research.google.com/github/eportah/Bird-of-Prey-Identifier/blob/main/updateModelPath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#setup dataset

#mount the drive to retrieve dataset
from google.colab import drive
drive.mount('/content/drive')

#import ZipFile to read dataset
from zipfile import ZipFile

#establish paths
zipPath = "/content/drive/MyDrive/MachineLearning/Datasets/birdsDatasets.zip"
extractPath = "/content/extractPath"

#extract and open dataset in read mode using ZipFile
with ZipFile(zipPath, 'r') as zipObj:
   zipObj.extractall(extractPath)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#load dataset

#cell output rundown
"""
- Dataset has 300 files (images) split into 2 classes.
- Class names are 'birdsOfPrey' and 'notBirdsOfPrey'.
- Images are batched into groups of 32, resized to 224x224 pixels, with 3 color channels RGB.
- Labels are stored in 1-D array of length 32, one label per image in the batch.
"""

#store dataset path into variable and import tensorflow to load images from directory
datasetDirectory = "/content/extractPath/birdsDataset"
import tensorflow as tf

#load dataset with function image_dataset_from_directory and establish both image size and batch size
trainDataset = tf.keras.utils.image_dataset_from_directory(datasetDirectory, image_size=(224,224), batch_size=32)

#preview what got loaded and what dataset looks like
class_names = trainDataset.class_names
print("Class names:", class_names)
for images, labels in trainDataset.take(1):
  print("Image batch shape:", images.shape)
  print("Label batch shape:", labels.shape)

In [ ]:
#split dataset

#cell output rundown
"""
- 210, 70% for training
- 90, 30% for validation
- Batch sizes are the same for both training and validation
"""

#load training dataset, 70% of data
trainingDataset = tf.keras.utils.image_dataset_from_directory(
   datasetDirectory,
   validation_split=0.3,
   subset="training",
   seed=123,
   image_size=(224,224),
   batch_size=32
)

#load validation dataset, 30% of data
validationDataset = tf.keras.utils.image_dataset_from_directory(
   datasetDirectory,
   validation_split=0.3,
   subset="validation",
   seed=123,
   image_size=(224,224),
   batch_size=32
)

#preview one batch from training dataset
print("Training batch shapes:")
for imagesTrain, labelsTrain in trainingDataset.take(1):
  print("Images:", imagesTrain.shape)
  print("Labels:", labelsTrain.shape)

#preview one batch from validation dataset
print("Validation batch shapes:")
for imagesVal, labelsVal in validationDataset.take(1):
 print("Images:", imagesVal.shape)
 print("Labels:", labelsVal.shape)

In [ ]:
#preprocess dataset

#import MobileNetV2 preprocessing
from keras.applications.mobilenet_v2 import preprocess_input

#define dataAugmentation
dataAugmentation = tf.keras.Sequential([
   tf.keras.layers.RandomFlip("horizontal"),
   tf.keras.layers.RandomRotation(0.1),
   tf.keras.layers.RandomZoom(0.1)
])

#apply augmentation and MobileNetV2 preprocessing to training set using map to transform batches of images
trainSet = trainingDataset.map(lambda x, y: (preprocess_input(dataAugmentation(x)),y))

#apply MobileNetV2 preprocessing to validation set still using map to transform batches of images
validSet = validationDataset.map(lambda x, y: (preprocess_input(x),y))

In [ ]:
#preview augmentation

#import pyplot to visualize tensors as images and numpy for numpy indexing
import matplotlib.pyplot as plt
import numpy as np

#take one raw image from unmapped training dataset and select first image and its label, shape (224, 224, 3)
for imgs, labs in trainingDataset.take(1):
  batchImages = imgs.numpy()
  batchLabels = labs.numpy()
sampleImage = batchImages[0]
sampleLabel = batchLabels[0]

#establish grid and create a figure for plotting
augmentedNum = 9
plt.figure(figsize = (6,6))

#add batch dimension and apply augmentation
for i in range(augmentedNum):
  imgBatch = np.expand_dims(sampleImage, axis = 0)
  imgAugmented = dataAugmentation(imgBatch)[0].numpy()

  #clip to ensure values are within range and cast to convert the aray from floats to 8-bit unsigned integers
  imgAugmented = np.clip(imgAugmented, 0, 255).astype("uint8")

  #finish off by plotting each augmented image in a 3x3 grid
  axes = plt.subplot(3,3, i+1)
  axes.imshow(imgAugmented)
  axes.axis('off')

In [ ]:
#build model

#import MobileNetV2 to use as base
from keras.applications import MobileNetV2
from keras import layers, models

#load base MobiletNetV2 and drop original classifier
baseModel = MobileNetV2(
    input_shape = (224, 224, 3),
    include_top = False,
    weights = 'imagenet'
)

#freeze base so pretrained weights are not trained right away
baseModel.trainable = False

#add custom classifier head by using layers
"""
Rectified linear unit relu to prevent exploding values
Softmax to turn output into probabilites that add up to 1, each value shows chances an image could belong to a class
Dense layer to learn how much each feature matters for classying a species with another
GAP2D to squeeze feature maps into single vector per channel instead of flattening huge tensors
Dropout to randomly turn off 30% neurons during training to prevent overfitting
"""
model = models.Sequential([
    baseModel,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation = 'relu'),
    layers.Dropout(0.3),
    layers.Dense(6, activation = 'softmax')
])

In [ ]:
#compile model

#import optimizers to use Adam
from keras import optimizers

#.compile for optimizer, loss, metrics
"""
Set learning rate to 0.0001 to avoid ruining pretrained weights
Sparse categorical crossentropy since classes are labeled as integers
Accuracy metric to track model performance
"""
model.compile(
    optimizer = optimizers.Adam(learning_rate = 0.0001),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
#train model

#cell output rundown
"""
Training data divided into batches of images (batch size = 32)
7 batches per epoch
Model updates its weights after each batch
Over epochs, accuracy increases and loss decreases
Validation accuracy starts high due to pretrained MobileNetV2 base
"""

#set 15 epochs to get started and change depending on how it fits
epochs = 15

#fit the model with training history
"""
history stores loss and accuracy curves to be plotted later
"""
history = model.fit(
    trainSet,
    validation_data = validSet,
    epochs = epochs
)

In [ ]:
#evaluate curves

#extract metrics from history
acc = history.history['accuracy']
valAcc = history.history['val_accuracy']
loss = history.history['loss']
valLoss = history.history['val_loss']

#number of epochs
epochsRange = range(len(acc))

#plot accuracy and loss
plt.figure(figsize=(12, 5))

#accuracy subplot
plt.subplot(1, 2, 1)
plt.plot(epochsRange, acc, label = 'Training Accuracy')
plt.plot(epochsRange, valAcc, label = 'Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

#loss subplot
plt.subplot(1, 2, 2)
plt.plot(epochsRange, loss, label = 'Training Loss')
plt.plot(epochsRange, valLoss, label = 'Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
#inspect predictions

#cell output rundown
"""
3x3 grid of 9 birds
Prediction compared with ground truth
If correctly predicted, values (1 or 0) align
birdsOfPrey, 1
notBirdsOfPrey, 0
"""

#take batch from validation set
"""
.predict to get probabilities
.argmax to get predicted class
axis=1 to look across columns for each row
actual (true) labels is ground truth
"""
for images, labels in validSet.take(1):
  predictions = model.predict(images)
  predictedLabels = np.argmax(predictions, axis=1)
  actualLabels = labels.numpy()
  imagesBatch = images.numpy()

#flip labels so that birdsOfPrey = 1
predictedLabels = 1 - predictedLabels
actualLabels = 1- actualLabels

#plot first 9 images in batch with prediction vs ground truth
"""
Show first 9 images in batch for simplicity
(images_batch[i]+1)/2 to convert from [-1,1] to [0,1]
Fstring to format labels
True for ground truth
.axis off to hide unwanted ticks and labels
"""
plt.figure(figsize=(6,6))
for i in range(9):
  axes = plt.subplot(3, 3, i+1)
  plt.imshow((imagesBatch[i]+1)/2)
  plt.title(f"Prediction: {predictedLabels[i]}, True: {actualLabels[i]}")
  plt.axis('off')

#tight layout so nothing overlaps
plt.tight_layout()
plt.show()

In [ ]:
#save model

#save to birdOfPreyIdentifier.keras
"""
.save to save completed model into permanent file
.keras as the file extension to save architecture, weights, and optimization state
File saved to temporary storage of Colab session
Saved permanently by downloading to desktop and uploading to mounted drive
"""
model.save('birdOfPreyIdentifier.keras')

#print statement to show save was succesful
print("Model saved to birdOfPreyIdentifier.keras")

In [13]:
#load model

#cell output rundown
"""
Layers
MobileNetV2 (base) extracts general image features
Custom layers take features from the base and learn how to use them for bird images

Output Shape
'None' represents flexible batch size, allows for training on large batches and predictions on individual items
Last shape means model outputs 6 prob scores for each image, one for each of the 6 bird classes

Parameters
Individual dials the model adjusts during training to learn
Param of 0 because they just perform a fixed math operation without learning anything
Trainable params are what was specifically tuned in project for bird dataset
Non-trainable params are frozen dials from pre-trained MobileNetV2 base that is being borrowed
Optimizer params are internal variables Adam uses to keep track of training process, not part of model's layers
Adam keeps track of its past average gradients and past average of the square of its gradients
"""

#import tf to make cell self-contained and avoid running extra cells
import tensorflow as tf

#establish path
modelPath = '/content/drive/MyDrive/MachineLearning/Models/birdOfPreyIdentifier.keras'

#load trained model
loadedModel = tf.keras.models.load_model(modelPath)

#print model summary to show load was succesful along with additional model info
loadedModel.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,752,212 (10.50 MB)

 Trainable params: 164,742 (643.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

 Optimizer params: 329,486 (1.26 MB)

In [ ]:
#predict unseen data

#prep for image preprocessing
import numpy as np
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import preprocess_input

#define class names used during training and define prediction function
"""
Load image from file path while resizing to match input size model was trained on
Convert image to np array to allow model to work with numbers rather than image files
Not using np.asarray() as it will be in uint8 rather than desired float32 with img_to_array
Modify shape by adding extra dimension to turn single image into batch of one
Set axis = 0 to insert new dimension at the very front of original shape (224,224,3)
Preprocess image by applying same scaling used during training
Get model's prediction to feed prepped image to loaded model and output array of probabilities
"""
classNames = ['birdsOfPrey', 'notBirdsOfPrey']
def predictBirdOfPrey(imagePath, model):

  img = image.load_img(imagePath, target_size = (224, 224))
  imgArray = image.img_to_array(img)
  imgBatch = np.expand_dims(imgArray, axis = 0)
  imgPreprocess = preprocess_input(imgBatch)
  prediction = model.predict(imgPreprocess)

  #establish prediction's human readibility and return for later use
  """
  Uses np.argmax to find index of highest probability
  Uses predicted index to look up the name in list
  Uses np.max to get highest probability and convert to percentage for confidence score
  Confidence score is probability that model assigns to its prediction for a single image
  """
  indexPredicted = np.argmax(prediction[0])
  classPredicted = classNames[indexPredicted]
  confidence = np.max(prediction[0]) * 100
  return classPredicted, confidence

#test function and use it to make prediction and print results with f-string
testImagePath = '/content/extractPath/birdsDataset/birdsOfPrey/eagle/eagleFlight1.jpg'
classPredicted, confidence = predictBirdOfPrey(testImagePath, loadedModel)
print(f"Prediction: {classPredicted}")
print(f"Confidence: {confidence:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Prediction: birdsOfPrey
Confidence: 99.55%


In [16]:
#install StreamLit by getting library

!pip install streamlit

In [17]:
#create app file

#import needed libraries and save cell content to new file called birdOfPreyApp.py
%%writefile birdOfPreyApp.py
import streamlit as st
import numpy as np
import tensorflow as tf
from PIL import Image
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import preprocess_input

#set up page
st.set_page_config(pageTitle="Bird Of Prey Identifier")
st.title("Bird of Prey Identifier")
st.write("Upload an image of a bird for the model to predict if it's a bird of prey or not.")

#streamlit cache command so app loads model once rather than each time an image is uploaded
@st.cache.resource

#load saved model and define prediction function
MODEL_PATH = 'birdOfPreyIdentifier.keras'
model = tf.keras.models.load_model(MODEL_PATH)
def loadTrainedModel():
  model = tf.keras.models.load_model(MODEL_PATH)
  return model

#define class names
CLASS_NAMES = ['eagle', 'falcon', 'hawk', 'owl', 'vulture', 'NotBirdsOfPrey']

#preprocess image and return predicted class and confidence score
"""
convert image to numpy array
add extra dimension for batch
preprocess image for model
get model's prediction and turn into human-readable result
get specific bird type
"""
def predictBirdOfPrey(img_to_predict, model):
  img = img_to_predict.convert('RGB')
  img = img.resize((224, 224))
  img_array = image.img_to_array(img)

  img_batch = np.expand_dims(img_array, axis=0)

  img_preprocessed = preprocess_input(img_batch)

  prediction = model.predict(img_preprocessed)
  predictedIndex = np.argmax(prediction[0])
  specificBird = CLASS_NAMES[predictedIndex]
  confidence = np.max(prediction[0]) * 100

  if specificBird == 'notBirdOfPrey':
    resultSpecBird = "Not a bird of prey"
  else:
    resultSpecBird = f"Bird of Prey ({specificBird.capitalize()})"

  return resultSpecBird, confidence

#load model and create file uploader
model = loadTrainedModel()
uploadedFile = st.file_uploader("Upload an image...", type=["jpg", "jpeg", "png"])

#classify uploaded image and yield prediction and confidence
"""
st.image to open file as an image
st.button to create Classify button
st.spinner for a loading message
st.success and st.info for results
"""
if uploadedFile is not None:
  pil_image = Image.open(uploadedFile)
  st.image(pil_image, caption='Uploaded image', use_column_width=True)
  if st.button('Classify'):
    with st.spinner('Classifying...'):
      predictedClass, confidence = predictBirdOfPrey(pil_image, model)
      st.success(f"Prediction: **{predictedClass}**")
      st.info(f"Confidence: **{confidence:.2f}%**")

Overwriting birdOfPreyApp.py


In [18]:
#deployment requirements

%%writefile requirements.txt
streamlit
tensorflow
Pillow
numpy

Overwriting requirements.txt
